In [ ]:
# Colab -> GitHub: sync .ipynb files

# --------- 0) MOUNT DRIVE ----------
from google.colab import drive
drive.mount('/content/drive')

# --------- 1) DEFAULTS (edit if needed) ----------
# Your Drive source folder
DRIVE_FOLDER = "/content/drive/MyDrive/colab_notebook_demo"  # your case

# Where to clone the repo inside Colab
REPO_DIR = "/content/repo"

# Subfolder path in the repo where notebooks should live
TARGET_PREFIX = "colab/demo_ipynb_notebooks"  # your case

# Fallback values (used only if git_info.txt not found)
GIT_USER_NAME = "Colab Sync"
GIT_USER_EMAIL = "colab-sync@example.com"
OWNER = "ai-for-dld"
REPO = "ai_for_dld_udemy"
BRANCH = "main"

# Optional locations to look for git_info.txt (first match wins)
GIT_INFO_CANDIDATES = [
    "/content/drive/MyDrive/git_info.txt",
    f"{DRIVE_FOLDER.rstrip('/')}/git_info.txt",
]

# --------- 2) TRY TO LOAD git_info.txt ----------
import os, re

def parse_git_info(path):
    kv = {}
    pat = re.compile(r'^\s*([A-Z_]+)\s*=\s*"(.*)"\s*$')
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            m = pat.match(line.strip())
            if m:
                kv[m.group(1)] = m.group(2)
    return kv

git_info_path = None
for cand in GIT_INFO_CANDIDATES:
    if os.path.exists(cand):
        git_info_path = cand
        break

if git_info_path:
    kv = parse_git_info(git_info_path)
    GIT_USER_NAME = kv.get("GIT_USER_NAME", GIT_USER_NAME)
    GIT_USER_EMAIL = kv.get("GIT_USER_EMAIL", GIT_USER_EMAIL)
    OWNER = kv.get("OWNER", OWNER)
    REPO = kv.get("REPO", REPO)
    BRANCH = kv.get("BRANCH", BRANCH)
    # Allow DRIVE_FOLDER / REPO_DIR / TARGET_PREFIX overrides if present
    DRIVE_FOLDER = kv.get("DRIVE_FOLDER", DRIVE_FOLDER)
    REPO_DIR = kv.get("REPO_DIR", REPO_DIR)
    TARGET_PREFIX = kv.get("TARGET_PREFIX", TARGET_PREFIX)
    print(f"Loaded settings from: {git_info_path}")
else:
    print("git_info.txt not found. Using defaults set in this cell.")

print("Settings:")
print("  OWNER:", OWNER)
print("  REPO:", REPO)
print("  BRANCH:", BRANCH)
print("  DRIVE_FOLDER:", DRIVE_FOLDER)
print("  REPO_DIR:", REPO_DIR)
print("  TARGET_PREFIX:", TARGET_PREFIX)

# --------- 3) TOKEN PROMPT ----------
from getpass import getpass
TOKEN = getpass("Enter a GitHub token with repo write access: ")

# --------- 4) CLONE OR UPDATE REPO ----------
import subprocess

def run(cmd, cwd=None):
    print(">", " ".join(cmd))
    subprocess.run(cmd, check=True, cwd=cwd)

REPO_URL = f"https://{TOKEN}@github.com/{OWNER}/{REPO}.git"

if not os.path.exists(REPO_DIR):
    run(["git", "clone", "--branch", BRANCH, REPO_URL, REPO_DIR])
else:
    # Ensure we are on the target branch and up to date
    run(["git", "fetch", "origin"], cwd=REPO_DIR)
    # If branch does not exist locally, create it to track origin/BRANCH
    try:
        run(["git", "checkout", BRANCH], cwd=REPO_DIR)
    except subprocess.CalledProcessError:
        run(["git", "checkout", "-b", BRANCH, f"origin/{BRANCH}"], cwd=REPO_DIR)
    run(["git", "pull", "--rebase", "origin", BRANCH], cwd=REPO_DIR)

# --------- 5) CONFIGURE GIT IDENTITY ----------
run(["git", "config", "user.name", GIT_USER_NAME], cwd=REPO_DIR)
run(["git", "config", "user.email", GIT_USER_EMAIL], cwd=REPO_DIR)

# --------- 6) COPY NOTEBOOKS PRESERVING STRUCTURE ----------
import shutil, glob

if not os.path.isdir(DRIVE_FOLDER):
    raise FileNotFoundError(f"DRIVE_FOLDER not found: {DRIVE_FOLDER}")

copied = 0
for path in glob.glob(os.path.join(DRIVE_FOLDER, "**", "*.ipynb"), recursive=True):
    rel = os.path.relpath(path, DRIVE_FOLDER).replace("\\", "/")
    dest = os.path.join(REPO_DIR, TARGET_PREFIX, rel) if TARGET_PREFIX else os.path.join(REPO_DIR, rel)
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy2(path, dest)
    copied += 1
    print("Copied:", rel)
print(f"Total notebooks copied: {copied}")

# --------- 7) STAGE ONLY .ipynb FILES ----------
ipynb_list = []
search_root = os.path.join(REPO_DIR, TARGET_PREFIX) if TARGET_PREFIX else REPO_DIR
for path in glob.glob(os.path.join(search_root, "**", "*.ipynb"), recursive=True):
    rel_repo = os.path.relpath(path, REPO_DIR).replace("\\", "/")
    ipynb_list.append(rel_repo)

if ipynb_list:
    # Add in manageable batches
    batch = 200
    for i in range(0, len(ipynb_list), batch):
        run(["git", "add"] + ipynb_list[i:i+batch], cwd=REPO_DIR)
else:
    print("No .ipynb files found to stage.")

# --------- 8) COMMIT AND PUSH ----------
try:
    run(["git", "commit", "-m", "Sync notebooks from Drive (Colab)"], cwd=REPO_DIR)
except subprocess.CalledProcessError:
    print("Nothing to commit (no changes).")

run(["git", "push", "origin", BRANCH], cwd=REPO_DIR)
print("Done.")
